# Notebook 04 — Optimización de Mantenimiento bajo Restricciones (Decisión)

**Objetivo:** transformar el riesgo (probabilidad de falla) en un plan de mantenimiento
óptimo bajo restricciones operativas (capacidad diaria, presupuesto, costos).

**Inputs:**
- Dataset: `data/processed/azure_pm/dataset_modelo.parquet`
- Modelo calibrado: `modelos/modelo_baseline_falla_30d.joblib`

**Outputs:**
- Plan diario recomendado (máquinas a intervenir)
- KPIs: ahorro esperado, costo de intervención, fallas capturadas (recall), Precision@K
- Archivos exportables a `data/processed/azure_pm/planes/`

In [1]:
from pathlib import Path
import numpy as np
import pandas as pd

SEED = 42
np.random.seed(SEED)

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 150)

In [5]:
import joblib

RAIZ = Path.cwd()
RUTA_DATASET = RAIZ / "data" / "processed" / "azure_pm" / "dataset_modelo.parquet"
RUTA_MODELO  = RAIZ / "modelos" / "modelo_baseline_falla_30d.joblib"

print("Working dir:", RAIZ)
print("Dataset existe:", RUTA_DATASET.exists(), "|", RUTA_DATASET)
print("Modelo existe :", RUTA_MODELO.exists(),  "|", RUTA_MODELO)

dataset_modelo = pd.read_parquet(RUTA_DATASET)

artefacto = joblib.load(RUTA_MODELO)
OBJETIVO = artefacto["objetivo"]
columnas_features = artefacto["columnas_features"]
modelo_calibrado = artefacto["modelo_calibrado"]

print("Dataset:", dataset_modelo.shape)
print("Objetivo:", OBJETIVO)

Working dir: c:\Users\sebas\OneDrive\Desktop\Proyecto Chatbot\Mantenimiento Industrial
Dataset existe: True | c:\Users\sebas\OneDrive\Desktop\Proyecto Chatbot\Mantenimiento Industrial\data\processed\azure_pm\dataset_modelo.parquet
Modelo existe : True | c:\Users\sebas\OneDrive\Desktop\Proyecto Chatbot\Mantenimiento Industrial\modelos\modelo_baseline_falla_30d.joblib
Dataset: (31567, 71)
Objetivo: falla_30d


In [6]:
dataset_modelo = dataset_modelo.sort_values(["fecha", "machineID"]).reset_index(drop=True)

ULTIMOS_DIAS = 7
fecha_max = dataset_modelo["fecha"].max()
fecha_min = fecha_max - pd.Timedelta(days=ULTIMOS_DIAS - 1)

ventana = dataset_modelo[(dataset_modelo["fecha"] >= fecha_min) & (dataset_modelo["fecha"] <= fecha_max)].copy()

print("Ventana:", str(fecha_min)[:10], "→", str(fecha_max)[:10])
print("Filas ventana:", ventana.shape)


Ventana: 2015-12-25 → 2015-12-31
Filas ventana: (75, 71)


In [7]:
X = ventana[columnas_features]
ventana["prob_falla"] = modelo_calibrado.predict_proba(X)[:, 1]

print("prob_falla: min/mean/max =",
      round(ventana["prob_falla"].min(), 2),
      round(ventana["prob_falla"].mean(), 2),
      round(ventana["prob_falla"].max(), 2))
ventana[["fecha", "machineID", "prob_falla"]].head()


prob_falla: min/mean/max = 0.43 0.59 0.76


,fecha,machineID,prob_falla
31492,2015-12-25,2,0.430739
31493,2015-12-25,15,0.529149
31494,2015-12-25,17,0.652922
31495,2015-12-25,20,0.651705
31496,2015-12-25,28,0.549421


In [8]:
riesgo_maquina = (
    ventana
    .groupby("machineID")
    .agg(
        fecha_ultima=("fecha", "max"),
        prob_mean=("prob_falla", "mean"),
        prob_max=("prob_falla", "max"),
        prob_p90=("prob_falla", lambda s: float(np.quantile(s, 0.90)))
    )
    .reset_index()
)

# score final (elige criterio)
# Recomendación: p90 o max, según tolerancia al riesgo
riesgo_maquina["prob_usada"] = riesgo_maquina["prob_p90"]

riesgo_maquina = riesgo_maquina.sort_values("prob_usada", ascending=False).reset_index(drop=True)

riesgo_maquina.head(10)


,machineID,fecha_ultima,prob_mean,prob_max,prob_p90,prob_usada
0,85,2015-12-26,0.737481,0.763037,0.757926,0.757926
1,15,2015-12-31,0.598505,0.711926,0.711926,0.711926
2,49,2015-12-26,0.705689,0.711926,0.710678,0.710678
3,54,2015-12-26,0.701379,0.711926,0.709816,0.709816
4,56,2015-12-25,0.699453,0.699453,0.699453,0.699453
5,20,2015-12-30,0.652296,0.699453,0.699453,0.699453
6,17,2015-12-27,0.679367,0.692590,0.692590,0.692590
7,28,2015-12-27,0.622321,0.699453,0.683180,0.683180
8,95,2015-12-31,0.635538,0.692590,0.668789,0.668789
9,83,2015-12-27,0.643888,0.652922,0.652922,0.652922


In [9]:
# Parámetros base (ajustables)
COSTO_FALLA_BASE = 120_000     # costo esperado típico de una falla (USD/CLP/UF abstracto)
COSTO_MANT_BASE  = 25_000      # costo mantención preventiva base
REDUCCION_RIESGO = 0.65        # mantención reduce probabilidad en 65%
RUIDO_COSTOS     = 0.20        # 20% variación por máquina

# Crear costos por máquina (reproducible)
rng = np.random.default_rng(SEED)

riesgo_maquina["factor_maquina"] = 1 + rng.normal(0, RUIDO_COSTOS, size=len(riesgo_maquina))
riesgo_maquina["factor_maquina"] = riesgo_maquina["factor_maquina"].clip(0.6, 1.6)

# costo_falla: base * factor
riesgo_maquina["costo_falla"] = (COSTO_FALLA_BASE * riesgo_maquina["factor_maquina"]).round(2)

# costo_mant: menor que falla, también varía y puede depender un poco del factor
riesgo_maquina["costo_mant"] = (COSTO_MANT_BASE * (0.9 + 0.2 * riesgo_maquina["factor_maquina"])).round(2)

# costo esperado sin intervención
riesgo_maquina["costo_esperado_sin_mant"] = (riesgo_maquina["prob_usada"] * riesgo_maquina["costo_falla"]).round(2)

# costo esperado con intervención: costo mant + prob reducida * costo falla
riesgo_maquina["prob_post_mant"] = (riesgo_maquina["prob_usada"] * (1 - REDUCCION_RIESGO)).round(4)
riesgo_maquina["costo_esperado_con_mant"] = (
    riesgo_maquina["costo_mant"] + riesgo_maquina["prob_post_mant"] * riesgo_maquina["costo_falla"]
).round(2)

# beneficio esperado (ahorro) de intervenir
riesgo_maquina["beneficio_esperado"] = (
    riesgo_maquina["costo_esperado_sin_mant"] - riesgo_maquina["costo_esperado_con_mant"]
).round(2)

riesgo_maquina[["machineID","prob_usada","costo_falla","costo_mant","costo_esperado_sin_mant","costo_esperado_con_mant","beneficio_esperado"]].head(10)


,machineID,prob_usada,costo_falla,costo_mant,costo_esperado_sin_mant,costo_esperado_con_mant,beneficio_esperado
0,85,0.757926,127313.21,27804.72,96493.95,61580.91,34913.04
1,15,0.711926,95040.38,26460.02,67661.68,50144.08,17517.60
2,49,0.710678,138010.83,28250.45,98081.31,62573.74,35507.57
3,54,0.709816,142573.55,28440.56,101201.04,63855.83,37345.21
4,56,0.699453,73175.16,25548.96,51182.58,43462.24,7720.34
5,20,0.699453,88747.69,26197.82,62074.83,47923.25,14151.58
6,17,0.692590,123068.17,27627.84,85235.79,57459.56,27776.23
7,28,0.683180,112410.18,27183.76,76796.38,54061.03,22735.35
8,95,0.668789,119596.77,27483.20,79985.00,55480.80,24504.20
9,83,0.652922,99526.95,26646.96,64983.29,49388.87,15594.42


In [10]:
CAPACIDAD_SEMANAL = 12     # máximo máquinas intervenidas en la semana
PRESUPUESTO = 350_000      # presupuesto total

print("CAPACIDAD_SEMANAL:", CAPACIDAD_SEMANAL)
print("PRESUPUESTO:", round(PRESUPUESTO, 2))

CAPACIDAD_SEMANAL: 12
PRESUPUESTO: 350000


In [12]:
# Intentar OR-Tools; si no está, mostramos mensaje claro
try:
    from ortools.linear_solver import pywraplp
except Exception as e:
    raise ImportError("Falta OR-Tools. Instala con: pip install ortools") from e


# Preparar datos para MILP
df_opt = riesgo_maquina.copy()

# Solo tiene sentido optimizar sobre máquinas con beneficio positivo
df_opt = df_opt[df_opt["beneficio_esperado"] > 0].reset_index(drop=True)
print("Candidatas con beneficio > 0:", len(df_opt))

solver = pywraplp.Solver.CreateSolver("SCIP")
if solver is None:
    raise RuntimeError("No se pudo crear solver SCIP. Prueba instalando OR-Tools correctamente.")

x = []
for i in range(len(df_opt)):
    x.append(solver.BoolVar(f"x_{int(df_opt.loc[i,'machineID'])}"))

# Restricción de capacidad
solver.Add(solver.Sum(x) <= CAPACIDAD_SEMANAL)

# Restricción de presupuesto
solver.Add(solver.Sum(x[i] * float(df_opt.loc[i, "costo_mant"]) for i in range(len(df_opt))) <= PRESUPUESTO)

# Objetivo: maximizar beneficio esperado total
solver.Maximize(solver.Sum(x[i] * float(df_opt.loc[i, "beneficio_esperado"]) for i in range(len(df_opt))))

status = solver.Solve()

if status != pywraplp.Solver.OPTIMAL:
    print("⚠️ No se encontró óptimo. Status:", status)
else:
    print("✅ Solución óptima encontrada.")


Candidatas con beneficio > 0: 17
✅ Solución óptima encontrada.


In [13]:
seleccion = []
for i in range(len(df_opt)):
    if x[i].solution_value() > 0.5:
        seleccion.append(i)

plan = df_opt.loc[seleccion].copy()
plan = plan.sort_values("beneficio_esperado", ascending=False).reset_index(drop=True)

# KPIs
n_seleccionadas = int(len(plan))
costo_mant_total = float(plan["costo_mant"].sum())
beneficio_total = float(plan["beneficio_esperado"].sum())

costo_sin = float(plan["costo_esperado_sin_mant"].sum())
costo_con = float(plan["costo_esperado_con_mant"].sum())

print("Máquinas seleccionadas:", n_seleccionadas)
print("Costo mant total:", round(costo_mant_total, 2))
print("Beneficio esperado total:", round(beneficio_total, 2))
print("Costo esperado sin mant:", round(costo_sin, 2))
print("Costo esperado con mant:", round(costo_con, 2))

plan[["machineID","fecha_ultima","prob_usada","costo_mant","costo_falla","beneficio_esperado"]].head(20)


Máquinas seleccionadas: 12
Costo mant total: 334068.52
Beneficio esperado total: 327866.01
Costo esperado sin mant: 1018370.53
Costo esperado con mant: 690504.52


,machineID,fecha_ultima,prob_usada,costo_mant,costo_falla,beneficio_esperado
0,54,2015-12-26,0.709816,28440.56,142573.55,37345.21
1,49,2015-12-26,0.710678,28250.45,138010.83,35507.57
2,85,2015-12-26,0.757926,27804.72,127313.21,34913.04
3,88,2015-12-30,0.652313,28379.40,141105.55,31451.20
4,64,2015-12-31,0.632790,28277.79,138667.01,28754.52
5,17,2015-12-27,0.692590,27627.84,123068.17,27776.23
6,90,2015-12-31,0.565865,28627.24,147053.79,25453.99
7,95,2015-12-31,0.668789,27483.20,119596.77,24504.20
8,28,2015-12-27,0.683180,27183.76,112410.18,22735.35
9,96,2015-12-29,0.623897,27566.03,121584.74,21736.23


In [14]:
DIRECTORIO_SALIDA = RAIZ / "data" / "processed" / "azure_pm"
DIRECTORIO_SALIDA.mkdir(parents=True, exist_ok=True)

ruta_plan = DIRECTORIO_SALIDA / "plan_mantencion_optimo.csv"
ruta_resumen = DIRECTORIO_SALIDA / "resumen_optimizacion.csv"

plan.to_csv(ruta_plan, index=False)

resumen = pd.DataFrame([{
    "fecha_max_dataset": str(fecha_max)[:10],
    "ultimos_dias": int(ULTIMOS_DIAS),
    "capacidad_semanal": int(CAPACIDAD_SEMANAL),
    "presupuesto": round(PRESUPUESTO, 2),
    "n_candidatas_beneficio_pos": int(len(df_opt)),
    "n_seleccionadas": int(n_seleccionadas),
    "costo_mant_total": round(costo_mant_total, 2),
    "beneficio_total": round(beneficio_total, 2),
    "costo_esperado_sin_mant_sel": round(costo_sin, 2),
    "costo_esperado_con_mant_sel": round(costo_con, 2),
    "reduccion_riesgo_param": round(REDUCCION_RIESGO, 2),
}])

resumen.to_csv(ruta_resumen, index=False)

print("✅ Plan guardado en:", ruta_plan.resolve())
print("✅ Resumen guardado en:", ruta_resumen.resolve())
resumen


✅ Plan guardado en: C:\Users\sebas\OneDrive\Desktop\Proyecto Chatbot\Mantenimiento Industrial\data\processed\azure_pm\plan_mantencion_optimo.csv
✅ Resumen guardado en: C:\Users\sebas\OneDrive\Desktop\Proyecto Chatbot\Mantenimiento Industrial\data\processed\azure_pm\resumen_optimizacion.csv


,fecha_max_dataset,ultimos_dias,capacidad_semanal,presupuesto,n_candidatas_beneficio_pos,n_seleccionadas,costo_mant_total,beneficio_total,costo_esperado_sin_mant_sel,costo_esperado_con_mant_sel,reduccion_riesgo_param
0,2015-12-31,7,12,350000,17,12,334068.52,327866.01,1018370.53,690504.52,0.65


In [15]:
def resolver_optimizacion(df_base, capacidad, presupuesto):
    # Reutiliza OR-Tools
    solver = pywraplp.Solver.CreateSolver("SCIP")
    x = [solver.BoolVar(f"x_{i}") for i in range(len(df_base))]

    solver.Add(solver.Sum(x) <= capacidad)
    solver.Add(solver.Sum(x[i] * float(df_base.loc[i, "costo_mant"]) for i in range(len(df_base))) <= presupuesto)

    solver.Maximize(solver.Sum(x[i] * float(df_base.loc[i, "beneficio_esperado"]) for i in range(len(df_base))))

    status = solver.Solve()
    if status != pywraplp.Solver.OPTIMAL:
        return None

    seleccion = [i for i in range(len(df_base)) if x[i].solution_value() > 0.5]
    plan = df_base.loc[seleccion]
    return {
        "capacidad": int(capacidad),
        "presupuesto": round(float(presupuesto), 2),
        "n_sel": int(len(plan)),
        "costo_mant_total": round(float(plan["costo_mant"].sum()), 2),
        "beneficio_total": round(float(plan["beneficio_esperado"].sum()), 2)
    }


grid_capacidad = [5, 8, 12, 16]
grid_presupuesto = [200_000, 300_000, 400_000, 500_000]

resultados = []
for cap in grid_capacidad:
    for pres in grid_presupuesto:
        out = resolver_optimizacion(df_opt.reset_index(drop=True), cap, pres)
        if out is not None:
            resultados.append(out)

sensibilidad = pd.DataFrame(resultados).sort_values(["capacidad","presupuesto"]).reset_index(drop=True)
sensibilidad


,capacidad,presupuesto,n_sel,costo_mant_total,beneficio_total
0,5,200000.0,5,141152.92,167971.54
1,5,300000.0,5,141152.92,167971.54
2,5,400000.0,5,141152.92,167971.54
3,5,500000.0,5,141152.92,167971.54
4,8,200000.0,7,197408.00,221201.76
5,8,300000.0,8,224891.20,245705.96
6,8,400000.0,8,224891.20,245705.96
7,8,500000.0,8,224891.20,245705.96
8,12,200000.0,7,197408.00,221201.76
9,12,300000.0,10,279640.99,290177.54


In [17]:
resultados = {
    "n_seleccionadas": int(len(plan)),
    "costo_mant_total": round(float(plan["costo_mant"].sum()), 2),
    "beneficio_total": round(float(plan["beneficio_esperado"].sum()), 2),
    "costo_sin_sel": round(float(plan["costo_esperado_sin_mant"].sum()), 2),
    "costo_con_sel": round(float(plan["costo_esperado_con_mant"].sum()), 2),
}
resultados


{'n_seleccionadas': 12,
 'costo_mant_total': 334068.52,
 'beneficio_total': 327866.01,
 'costo_sin_sel': 1018370.53,
 'costo_con_sel': 690504.52}

# Conclusiones — Notebook 04

En este notebook se implementó un módulo de decisión que transforma el output del modelo predictivo
(probabilidad de falla calibrada) en un **plan de mantención bajo restricciones operativas**.

## Enfoque implementado

1. **Scoring**: se estimó el riesgo de falla por máquina utilizando el modelo calibrado (`falla_30d`).
2. **Agregación operativa**: se consolidó el riesgo a nivel máquina para una ventana reciente de operación.
3. **Modelo de costos**: se tradujo el riesgo a **costo esperado** (falla vs mantención preventiva), incorporando
   parámetros de reducción de riesgo post-mantención y heterogeneidad por máquina.
4. **Optimización bajo restricciones**: se seleccionó el conjunto de máquinas a intervenir maximizando
   el **beneficio esperado** (ahorro), sujeto a capacidad y presupuesto.

## Resultados del plan

Bajo una capacidad máxima de **12 máquinas** y el presupuesto configurado, el plan resultante fue:

- **Máquinas seleccionadas:** 12  
- **Costo total de mantención preventiva:** 334.068,52  
- **Beneficio esperado total (ahorro):** 327.866,01  

Comparación de costo esperado sobre las máquinas seleccionadas:

- **Costo esperado sin intervención:** 1.018.370,53  
- **Costo esperado con intervención:** 690.504,52  
- **Reducción de costo esperado:** 327.866,01  

Estos resultados muestran una planificación consistente: el algoritmo prioriza intervenciones con mayor
valor económico esperado, manteniéndose dentro de las restricciones definidas.

## Lectura operacional

- El plan consume prácticamente todo el presupuesto disponible, lo que indica que existían suficientes
  oportunidades con beneficio positivo para justificar intervención dentro del horizonte.
- El beneficio esperado es comparable al gasto en mantención (relación cercana a 1:1), lo que sugiere
  que el umbral de intervención está en una zona donde cada mantención “se paga sola” en expectativa.

## Próximos pasos

1. Incorporar costos reales (por tipo de falla, componente o criticidad) si están disponibles.
2. Integrar restricciones adicionales: “cooldown” por máquina, ventanas de producción, prioridades de planta.
3. Ejecutar análisis de sensibilidad (capacidad/presupuesto) para construir una curva de trade-off.
4. Automatizar el flujo en `main.py` para generar el plan de forma recurrente y versionada.